In [1]:
# !pip install pygame

In [2]:
import gym
import numpy as np
from itertools import count
from collections import namedtuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [3]:
# for colab pygame compatibility
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"

In [4]:
# Hyperparamerters
gamma = 0.99
seed = 543
render = True
log_interval = 10

# Cart Pole
env = gym.make('CartPole-v1')
env.reset(seed=seed)
torch.manual_seed(seed)

SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])


class Policy(nn.Module):
    """
    implements both actor and critic in one model
    """
    def __init__(self):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(4, 128)

        # actor's layer
        self.action_head = nn.Linear(128, 2)

        # critic's layer
        self.value_head = nn.Linear(128, 1)

        # action & reward buffer
        self.saved_actions = []
        self.rewards = []

    def forward(self, x):
        """
        forward of both actor and critic
        """
        x = F.relu(self.fc1(x))

        # actor: choses action to take from state s_t
        # by returning probability of each action
        action_prob = F.softmax(self.action_head(x), dim=-1)

        # critic: evaluates being in the state s_t
        state_values = self.value_head(x)

        # return values for both actor and critic as a tuple of 2 values:
        # 1. a list with the probability of each action over the action space
        # 2. the value from state s_t
        return action_prob, state_values


model = Policy()
optimizer = optim.Adam(model.parameters(), lr=5+e-3)
# eps is for not making 0 denominator
eps = np.finfo(np.float32).eps.item()


/usr/local/lib/python3.8/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.8/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [5]:
def select_action(state):
    state = torch.from_numpy(state).float()
    probs, state_value = model(state)

    # create a categorical distribution over the list of probabilities of actions
    m = Categorical(probs)

    # and sample an action using the distribution
    action = m.sample()

    # save to action buffer
    model.saved_actions.append(SavedAction(m.log_prob(action), state_value))

    # the action to take (left or right)
    return action.item()

In [6]:
def train():
    """
    Training code. Calculates actor and critic loss and performs backprop.
    """
    R = 0
    saved_actions = model.saved_actions
    policy_losses = [] # list to save actor (policy) loss
    value_losses = [] # list to save critic (value) loss
    returns = [] # list to save the true values

    # calculate the true value using rewards returned from the environment
    for r in model.rewards[::-1]:
        # calculate the discounted value
        R = r + gamma * R
        returns.insert(0, R)

    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)

    for (log_prob, value), R in zip(saved_actions, returns):
        advantage = R - value.item()

        # calculate actor (policy) loss
        policy_losses.append(-log_prob * advantage)

        # calculate critic (value) loss using L1 smooth loss
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([R])))

    # reset gradients
    optimizer.zero_grad()

    # sum up all the values of policy_losses and value_losses
    loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()

    # backpropagation
    loss.backward()
    # update weights in layers
    optimizer.step()

    # reset rewards and action buffer
    del model.rewards[:]
    del model.saved_actions[:]    

In [7]:
def main():
    running_reward = 10

    for i_episode in count(1):

        # reset environment and episode reward
        state = env.reset()
        ep_reward = 0

        # for each episode, only run limited steps so that we don't
        # infinite loop while learning
        for t in range(1, 400):

            # select action from policy
            action = select_action(state)

            # take the action
            state, reward, done, _ = env.step(action)

            if render:
                env.render()

            model.rewards.append(reward)
            ep_reward += reward
            if done:
                break

        # update cumulative reward
        running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward

        # perform backprop
        train()

        # log results
        if i_episode % log_interval == 0:
            print(f'Episode {i_episode}\tLast reward: {ep_reward:.2f}\tAverage reward: {running_reward:.2f}')

        # check if we have "solved" the cart pole problem
        if running_reward > env.spec.reward_threshold:
            print(f"Solved! Running reward is now {running_reward} and "
                  f"the last episode runs to {t} time steps!")
            break

In [ ]:
main()

/usr/local/lib/python3.8/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


Episode 10	Last reward: 18.00	Average reward: 11.76
Episode 20	Last reward: 19.00	Average reward: 12.81
Episode 30	Last reward: 39.00	Average reward: 17.37
Episode 40	Last reward: 47.00	Average reward: 24.35
Episode 50	Last reward: 20.00	Average reward: 30.64
Episode 60	Last reward: 76.00	Average reward: 43.65
Episode 70	Last reward: 59.00	Average reward: 47.95
Episode 80	Last reward: 41.00	Average reward: 45.93
Episode 90	Last reward: 114.00	Average reward: 75.46
Episode 100	Last reward: 191.00	Average reward: 99.55
Episode 110	Last reward: 133.00	Average reward: 136.82
Episode 120	Last reward: 126.00	Average reward: 182.63
Episode 130	Last reward: 92.00	Average reward: 177.83
Episode 140	Last reward: 119.00	Average reward: 171.12
Episode 150	Last reward: 399.00	Average reward: 188.42
Episode 160	Last reward: 399.00	Average reward: 217.20
Episode 170	Last reward: 399.00	Average reward: 275.83
Episode 180	Last reward: 399.00	Average reward: 311.97
Episode 190	Last reward: 229.00	Averag